# Problema

    Dado uma base de localizações de 1) empresas que precisam realizar entrega de seus produtos e de 2) entregadores disponíveis para realizar essas entregas, como designar cada entrega para cada entregador de forma a otimizar a logística?

# Hipótese

    Se usarmos técnicas de clusterização, podemos identificar os grupos de entregas ou grupos de entregas + entregadores e assim definir clusters de entregas por proximidade.
    
## Proposta 1:

    Definindo a localização do entregador como centróide do cluster, buscando as entregas mais próximas a ele

## Proposta 2:

    Definir clusters ustilizando apenas as localizações do estabelecimentos.
    Após isso, linkar o entregador ao cluster, a partir da entrega mais próxima dele.
    
## Proposta 3:

    Definir clusters com clientes e entregdores juntos.
    E depois uma subclusterização de cada cluster com mais de um entregador.
    

### Cenário 1:

    Número de entregas > Número de entregadores
    
 ### Cenário 2:

    Número de entregas < Número de entregadores

In [68]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, KMeans
import plotly.express as px

In [262]:
geolocations = pd.read_csv('.\\Bases\\olistbr-brazilian-ecommerce\\olist_geolocation_dataset.csv')
customers = pd.read_csv('.\\Bases\\olistbr-brazilian-ecommerce\\olist_customers_dataset.csv')
sellers = pd.read_csv('.\\Bases\\olistbr-brazilian-ecommerce\\olist_sellers_dataset.csv')

In [263]:
print(geolocations.shape)
print(customers.shape)
print(sellers.shape)

(1000163, 5)
(99441, 5)
(3095, 4)


In [264]:
geolocations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [265]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [266]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [141]:
df = (
    geolocations
    .groupby("geolocation_state")
    .agg(count = pd.NamedAgg(column="geolocation_zip_code_prefix", aggfunc="count"),
         countD = pd.NamedAgg(column="geolocation_zip_code_prefix", aggfunc=pd.Series.nunique),
        )
    .reset_index()
)

fig1 = px.bar(data_frame=df.sort_values(by="count", ascending=False),
              x="geolocation_state",
              y="count")

fig2 = px.bar(data_frame=df.sort_values(by="countD", ascending=False),
              x="geolocation_state",
              y="countD")

fig1.show()
fig2.show()

# Tratamento dos dados
- Optei por usar apenas `geolocation_state == "SP"` 
- Existem `geolocation_zip_code_prefix` repetidos

State  | Count            | Count Distinct | %
------ | ---------------- | -------------- | ----
All    | 1,000,163        | 19,015         | 1.90
SP     | 404,268          | 6,349          | 1.57

- porém com `geolocation_lat` e `geolocation_lng` muito próximas. Ao se verificar o desvio padrão dessas coordenadas para o mesmo id, o resultado foi majoritariamente abaixo de 0.1.
    - Dito isso, decidi agrupar por id e utilizar a média das coordenadas
- Também existem `geolocation_city` iguais com acentuações diferentes
    - Removi todos as acentuações
    - ao tentar resolver esse problema, reparei que esta coluna e a geolocation_state possuem alguns valores incoerentes.
        - porém para SP eram apenas 556 registros, que equivale à 0.13% do total. Decidi então apenas removê-los da análise

### Verificando Granularidade da base

In [147]:
geolocations2 = geolocations[geolocations["geolocation_state"] == "SP"]

In [148]:
print(geolocations.count()[0])
print(geolocations.drop_duplicates("geolocation_zip_code_prefix").count()[0])

1000163
19015


In [149]:
print(geolocations2.count()[0])
print(geolocations2.drop_duplicates("geolocation_zip_code_prefix").count()[0])

404268
6349


In [152]:
df = (
    geolocations2.groupby("geolocation_zip_code_prefix")
    .agg(count = pd.NamedAgg(column="geolocation_zip_code_prefix", aggfunc="count"))
    .reset_index()
)
df[df["count"] > 1].sort_values(by=["count"], ascending=False)

,geolocation_zip_code_prefix,count
4501,11680,879
4528,11740,788
4381,11250,602
3610,7600,587
4696,12460,561
...,...,...
3975,8666,2
3427,6869,2
6287,19505,2
5607,15617,2


In [257]:
df[df["count"]==1]

,geolocation_zip_code_prefix,count
90,1189,1
91,1200,1
151,1262,1
152,1290,1
297,2009,1
...,...,...
6258,19140,1
6274,19295,1
6285,19490,1
6289,19540,1


In [258]:
geolocations2[geolocations2["geolocation_zip_code_prefix"] == 1189]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
3282,1189,-23.532108,-46.638395,sao paulo,SP


In [260]:
customers[customers["customer_zip_code_prefix"] == 1189]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [261]:
sellers[sellers["seller_zip_code_prefix"] == 1189]

,seller_id,seller_zip_code_prefix,seller_city,seller_state


In [167]:
order_items[order_items["seller_id"]=="e7f3bf1ecd8526eb7f3b18059e6716e5"]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
4199,0977997b1c07196d7d76a0d79c50117d,1,17e7128a33dae59e32be411b7b4921b5,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-03-28 18:42:09,19.99,10.96
6997,0fd8a0c97993a139d5277f2023f93649,1,af1190b9fff9a3c0ea7bc29ef8dfd8c1,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-04-13 17:03:18,99.90,14.87
12731,1cd8d1567debd198821fe9d46d99b5b1,1,af1190b9fff9a3c0ea7bc29ef8dfd8c1,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-09-29 20:28:20,89.50,15.38
23180,34e6b9f685df8e8b6465a715689c749f,1,4254fb9a475231dc0442038cd846a387,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-05-10 10:15:20,97.40,25.17
23348,353c2d85938ff4228ec749b7b461561b,1,17e7128a33dae59e32be411b7b4921b5,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-07-12 21:23:05,32.70,11.85
23349,353c2d85938ff4228ec749b7b461561b,2,17e7128a33dae59e32be411b7b4921b5,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-07-12 21:23:05,32.70,11.85
41905,5f783382a74cf95c03c789405c2fb161,1,677ea65357366d30a013af105b2c8a0f,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-04-19 08:55:12,99.90,14.87
43106,620f88e1cad4c60e7f3cc29d682e7eb9,1,3d07b18589d2af2cfb638c3dbc8ff826,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-04-13 04:42:55,94.50,14.83
52711,77f120015ca363b1f0bd4c879f902eab,2,99306a16cc05ea73566ac3a2d92122ce,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-04-06 16:42:26,99.90,9.89
52954,78886227da2df65c57f57d2fec577bce,1,96cc6d39d2114a36f318d641c3cae43d,e7f3bf1ecd8526eb7f3b18059e6716e5,2017-04-13 17:15:13,99.90,11.31


In [164]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [ ]:
geolocations3 = (
    geolocations2
    .groupby(["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"])
    .agg(geolocation_lat=pd.NamedAgg(column="geolocation_lat", aggfunc="mean"),
         geolocation_lng=pd.NamedAgg(column="geolocation_lng", aggfunc="mean")
        )
    .reset_index()
)

In [150]:
geolocations3.count()

geolocation_zip_code_prefix    10671
geolocation_city               10671
geolocation_state              10671
geolocation_lat                10671
geolocation_lng                10671
dtype: int64

In [276]:
# df = (geolocations2.groupby("geolocation_zip_code_prefix")
#  .agg({"geolocation_zip_code_prefix":"count"})
#  .rename(columns={"geolocation_zip_code_prefix":"count"})
#  .reset_index()
# )
df[df["count"] > 1].sort_values(by=["count"], ascending=False)
# geolocations2[geolocations2.isin({"geolocation_zip_code_prefix":[1307, 2116, 3203]})["geolocation_zip_code_prefix"]]

,geolocation_zip_code_prefix,count
4501,11680,879
4528,11740,788
4381,11250,602
3610,7600,587
4696,12460,561
...,...,...
3975,8666,2
3427,6869,2
6287,19505,2
5607,15617,2


In [292]:
geolocations2[geolocations2["geolocation_zip_code_prefix"] == 1307].describe()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
count,311.0,311.000000,311.000000
mean,1307.0,-23.552778,-46.653208
std,0.0,0.003609,0.003474
min,1307.0,-23.558604,-46.697678
25%,1307.0,-23.554081,-46.654376
50%,1307.0,-23.552836,-46.653169
75%,1307.0,-23.551762,-46.651555
max,1307.0,-23.502632,-46.646325


### Ajustando caracteres especiais de coluna `geolocation_city` 

In [ ]:
geolocations2 = pd.DataFrame.copy(geolocations)
geolocations2["geolocation_city"] = geolocations2["geolocation_city"].apply(unidecode)

### Ajustando colunas `geolocation_lat` e `geolocation_lng`

In [117]:
states_geoloc = (
    geolocations
    .groupby("geolocation_state")
    .agg(countD = pd.NamedAgg(column="geolocation_zip_code_prefix", aggfunc=pd.Series.nunique))
#     .agg({"geolocation_zip_code_prefix":"count"})
#     .rename(columns={"geolocation_zip_code_prefix":"count"})
    .sort_values(by="countD", ascending=False)
    .reset_index()
)

In [99]:
556/404268*100

0.1375325279270187

In [7]:
sellers["origin"] = 1
customers["destination"] = 1

In [8]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,destination
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,1
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,1
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,1
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,1


In [9]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state,origin
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,1
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,1
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,1
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,1
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,1


In [10]:
base = (
    geolocations[geolocations["geolocation_state"]=="SP"]
    .rename(columns={"geolocation_zip_code_prefix":"id"})
    .join(
        (customers
         .rename(columns={"customer_zip_code_prefix":"id"})
         [["id", "destination"]]
         .set_index("id")),
        on="id",
    )
    .join(
        (sellers
         .rename(columns={"seller_zip_code_prefix":"id"})
         [["id", "origin"]]
         .set_index("id")),
        on="id"
    )
)

In [35]:
base[base["id"]==1001].groupby("id").agg({"geolocation_lat":["mean", "std"], "geolocation_lng":["mean", "std"]})

geolocation_lat           geolocation_lng          
                mean       std            mean       std
id                                                      
1001       -23.55019  0.000698      -46.634024  0.000303

In [97]:
base2 = (
    base
    .groupby("id")
    .agg(lat_mean=pd.NamedAgg(column="geolocation_lat", aggfunc="mean"),
         lat_std=pd.NamedAgg(column="geolocation_lat", aggfunc="std"),
         lng_mean=pd.NamedAgg(column="geolocation_lng", aggfunc="mean"),
         lng_std=pd.NamedAgg(column="geolocation_lng", aggfunc="std"),
        )
    .reset_index()
)
base2["lat_std"] = np.round(base2["lat_std"], 4)
base2["lng_std"] = np.round(base2["lng_std"], 4)
# base2 = (base2
#          .groupby("id")
#          .agg(
#              mean_lat_std=pd.NamedAgg(column="lat_std", aggfunc="mean"),
#              mean_lng_std=pd.NamedAgg(column="lng_std", aggfunc="mean")
#          )
#         )
# plt.hist(base2["mean_lng_std"], bins=100, range=(-0.25,0.25))
base2[(base2["lat_std"] > 0.01) | (base2["lng_std"] > 0.01)]
# base2[base2.isin({"id":[1307, 2116, 3203]})["id"]]

,id,lat_mean,lat_std,lng_mean,lng_std
70,1131,-23.522380,0.0053,-46.643108,0.0234
130,1239,-23.543095,0.0116,-46.631059,0.0657
135,1244,-23.562049,0.0399,-46.659590,0.0073
202,1417,-23.552779,0.0260,-46.633932,0.0791
215,1430,-23.656140,0.1416,-46.662502,0.0123
...,...,...,...,...,...
6320,19840,-22.615059,0.0104,-50.668082,0.0083
6321,19845,-24.007627,1.3860,-50.477272,2.0332
6322,19860,-22.743476,0.0026,-50.787861,0.0110
6325,19880,-22.751129,0.0184,-50.386729,0.0125
